In [5]:
import os
import json
import sqlite3
from tqdm import tqdm

# 📁 1. Dossier des JSON
folder_path = "C:/Users/pierr/OneDrive/Bureau/B3/Projet_python/Data_ynov_projet/test_jp.json" # ⚠️ Change si besoin
db_path = "C:/Users/pierr/OneDrive/Bureau/test.db"

# 📦 2. Connexion SQLite + protection concurrente
conn = sqlite3.connect(db_path, timeout=30)
conn.execute("PRAGMA journal_mode=WAL;")
cursor = conn.cursor()


In [6]:

# 🧱 3. Création du schéma relationnel
cursor.executescript("""
CREATE TABLE IF NOT EXISTS Playlist (
    pid INTEGER PRIMARY KEY,
    name TEXT,
    collaborative BOOLEAN,
    modified_at BIGINT,
    num_tracks INTEGER,
    num_albums INTEGER,
    num_followers INTEGER
);

CREATE TABLE IF NOT EXISTS Artist (
    artist_uri TEXT PRIMARY KEY,
    artist_name TEXT
);

CREATE TABLE IF NOT EXISTS Album (
    album_uri TEXT PRIMARY KEY,
    album_name TEXT
);

CREATE TABLE IF NOT EXISTS Track (
    track_uri TEXT PRIMARY KEY,
    track_name TEXT,
    duration_ms INTEGER,
    artist_uri TEXT,
    album_uri TEXT,
    FOREIGN KEY (artist_uri) REFERENCES Artist(artist_uri),
    FOREIGN KEY (album_uri) REFERENCES Album(album_uri)
);

CREATE TABLE IF NOT EXISTS PlaylistTrack (
    playlist_id INTEGER,
    track_uri TEXT,
    position INTEGER,
    PRIMARY KEY (playlist_id, position),
    FOREIGN KEY (playlist_id) REFERENCES Playlist(pid),
    FOREIGN KEY (track_uri) REFERENCES Track(track_uri)
);
""")
conn.commit()


In [7]:

# 🧮 4. Lecture de tous les fichiers JSON
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]
print(f"{len(json_files)} fichiers trouvés dans {folder_path}")

NotADirectoryError: [WinError 267] Nom de répertoire non valide: 'C:/Users/pierr/OneDrive/Bureau/B3/Projet_python/Data_ynov_projet/test_jp.json'

In [ ]:

# 📥 5. Insertion dans les tables
for filename in tqdm(json_files, desc="Importation JSON → SQL"):
    try:
        with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as f:
            data = json.load(f)

        for playlist in data.get("playlists", []):
            pid = playlist["pid"]

            # ✅ Playlist
            cursor.execute("""
                INSERT OR IGNORE INTO Playlist (
                    pid, name, collaborative, modified_at,
                    num_tracks, num_albums, num_followers
                ) VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                pid,
                playlist.get("name"),
                playlist.get("collaborative") == "true",
                playlist.get("modified_at"),
                playlist.get("num_tracks"),
                playlist.get("num_albums"),
                playlist.get("num_followers")
            ))

            for track in playlist.get("tracks", []):
                # ✅ Artist
                cursor.execute("""
                    INSERT OR IGNORE INTO Artist (artist_uri, artist_name)
                    VALUES (?, ?)
                """, (
                    track.get("artist_uri"),
                    track.get("artist_name")
                ))

                # ✅ Album
                cursor.execute("""
                    INSERT OR IGNORE INTO Album (album_uri, album_name)
                    VALUES (?, ?)
                """, (
                    track.get("album_uri"),
                    track.get("album_name")
                ))

                # ✅ Track
                cursor.execute("""
                    INSERT OR IGNORE INTO Track (
                        track_uri, track_name, duration_ms,
                        artist_uri, album_uri
                    ) VALUES (?, ?, ?, ?, ?)
                """, (
                    track.get("track_uri"),
                    track.get("track_name"),
                    track.get("duration_ms"),
                    track.get("artist_uri"),
                    track.get("album_uri")
                ))

                # ✅ PlaylistTrack
                cursor.execute("""
                    INSERT OR IGNORE INTO PlaylistTrack (
                        playlist_id, track_uri, position
                    ) VALUES (?, ?, ?)
                """, (
                    pid,
                    track.get("track_uri"),
                    track.get("pos")
                ))

    except Exception as e:
        print(f"❌ Erreur avec {filename} : {e}")

# ✅ 6. Commit final
conn.commit()

# 📊 7. Statistiques
cursor.execute("SELECT COUNT(*) FROM Playlist"); print("Playlists :", cursor.fetchone()[0])
cursor.execute("SELECT COUNT(*) FROM Track"); print("Tracks :", cursor.fetchone()[0])
cursor.execute("SELECT COUNT(*) FROM Artist"); print("Artists :", cursor.fetchone()[0])
cursor.execute("SELECT COUNT(*) FROM Album"); print("Albums :", cursor.fetchone()[0])
cursor.execute("SELECT COUNT(*) FROM PlaylistTrack"); print("Relations Playlist-Track :", cursor.fetchone()[0])

conn.close()
print("\n🎉 Import terminé avec succès !")
